## Imports and Defs

### Imports

In [171]:
import requests
from bs4 import BeautifulSoup
import zipfile
import os
import io

from urllib.parse import urljoin
from io import BytesIO
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)

import unidecode
import string

import time
import datetime
import pickle


### Variables

In [2]:
prefix = 'dfp'
radical = 'cia_aberta'
extension = '.zip'
demo_cvmnstracoes_financeiras = ['DRA', 'DMPL', 'DFC_MD', 'DFC_MI', 'BPA', 'BPP', 'DRE', 'DVA']  # Add all other items
demo_cvmnstracoes_financeiras_dict = {
    'BPA': 'Balanço Patrimonial Ativo (BPA)',
    'BPP': 'Balanço Patrimonial Passivo (BPP)',
    'DFC_MD': 'demo_cvmnstração de Fluxo de Caixa - Método Direto (DFC-MD)',
    'DFC_MI': 'demo_cvmnstração de Fluxo de Caixa - Método Indireto (DFC-MI)',
    'DMPL': 'demo_cvmnstração de Mutações do Patrimônio Líquido (DMPL)',
    'DRA': 'demo_cvmnstração de Resultado Abrangente (DRA)',
    'DRE': 'demo_cvmnstração de Resultado (DRE)',
    'DVA': 'demo_cvmnstração de Valor Adicionado (DVA)'
}
base_de_consolidacao = ['ind', 'con']  # Add other variables if existing
base_de_consolidacao_dict = {
    'ind': 'Individual',
    'con': 'Consolidado'
}



In [3]:
# URL base do site
base_cvm = "https://dados.cvm.gov.br/dados/CIA_ABERTA/"

# Inicializar uma sessão
session = requests.Session()

# Lista para armazenar links de arquivos CSV e ZIP
files_list = []

# Conjunto para armazenar subpastas já visitadas
visited_subfolders = set()



In [4]:
col_category = ['FILE_NAME', 'demonstrativo_cvm', 'BALANCE_SHEET', 'ANO', 'AGRUPAMENTO', 'CNPJ_CIA', 'VERSAO', 'DENOM_CIA', 'CD_CVM', 'GRUPO_DFP', 'MOEDA', 'ESCALA_MOEDA', 'ORDEM_EXERC', 'CD_CONTA', 'DS_CONTA', 'ST_CONTA_FIXA', 'COLUNA_DF']
col_datetime = ['DT_REFER', 'DT_FIM_EXERC', 'DT_INI_EXERC']
col_float = ['VL_CONTA']



### Defs

In [5]:
# save and load pkl
def save_pkl(demo_cvm, file_name):
    # Save dump pickle
    with open(f'{file_name}.pkl', 'wb') as f:
        pickle.dump(demo_cvm, f)
    return demo_cvm

def load_pkl(file_name):
    # Read load pickle
    with open(f'{file_name}.pkl', 'rb') as f:
        demo_cvm = pickle.load(f)
    return demo_cvm

In [6]:
def remaining_time(start_time, size, i):
    # Calculate the number of items processed and the number of remaining items
    counter = i + 1
    remaining_items = size - counter
    
    # Calculate the percentage of completion
    percentage = counter / size
    
    # Calculate the elapsed time
    running_time = time.time() - start_time
    
    # Calculate the average time taken per item
    avg_time_per_item = running_time / counter
    
    # Calculate the remaining time based on the average time per item
    remaining_time = remaining_items * avg_time_per_item
    
    # Convert remaining time to hours, minutes, and seconds
    hours, remainder = divmod(int(remaining_time), 3600)
    minutes, seconds = divmod(remainder, 60)
    
    # Format remaining time as a string
    remaining_time_formatted = f'{int(hours)}h {int(minutes):02}m {int(seconds):02}s'
    
    # Create a progress string with all the calculated values
    progress = (
        f'{percentage:.2%} '
        f'{counter}+{remaining_items}, '
        f'{avg_time_per_item:.6f}s per item, '
        f'Remaining: {remaining_time_formatted}'
    )

    return progress

In [7]:
# Função auxiliar para reunir links
def gather_links(url):
    visited_subfolders.add(url)  # Marcar a subpasta como visitada
    response = session.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, "html.parser")

    for link in soup.find_all("a"):
        href = link.get("href")
        full_link = urljoin(url, href)

        if full_link.startswith(base_cvm) and full_link not in visited_subfolders:
            if full_link.endswith((".csv", ".zip", ".txt")):
                files_list.append(full_link)
            elif full_link.endswith("/"):
                gather_links(full_link)
    return files_list



In [8]:
# helper for categories
def get_categories(files_list):
    # Extrair categorias e arquivos meta
    categories = set()
    meta_files = [file_link for file_link in files_list if "meta" in file_link]
    files = [file_link for file_link in files_list if "meta" not in file_link]

    for file_link in files_list:
        cat = '/'.join(file_link.replace(base_cvm,'').split('/')[:-2])
        categories.add(cat)
    categories = sorted(list(categories))

    return categories


In [9]:
# Função auxiliar para extrair metadados
def extract_meta(content):
    meta_dict = {}
    blocks = content.split("-----------------------\r\nCampo: ")[1:]
    
    for block in blocks:
        lines = block.strip().split("\r\n")
        campo = lines[0]
        descricao = None
        
        for line in lines:
            if 'Descrição' in line or 'Descrio' in line:
                descricao = line.split(':')[1].strip()
                break
        
        if descricao is not None:
            meta_dict[campo] = descricao
    
    return meta_dict



In [10]:
# Extrair e processar metadados
def get_metadados(files_list):
    meta_dict = {}
    meta_files = [file_link for file_link in files_list if "meta" in file_link]

    for file in meta_files:
        response = session.get(file)
        response.raise_for_status()

        if file.endswith('.zip'):
            zip_file = zipfile.ZipFile(io.BytesIO(response.content))

            for file_in_zip in zip_file.namelist():
                with zip_file.open(file_in_zip) as zip_file_content:
                    file_content = zip_file_content.read().decode('utf-8', errors='ignore')
                    d = extract_meta(file_content)
                    meta_dict[file_in_zip.split('.')[0]] = d
        elif file.endswith('.txt'):
            file_content = response.content.decode('iso-8859-1')
            d = extract_meta(file_content)
            file_name = file.split('/')[-1].split('.')[0]
            meta_dict[file_name] = d

    return meta_dict

In [11]:
# extract financial sheets from web zip files
def download_database(demonstrativos_cvm):
    # Initialize variables
    fin_sheet = []
    fin_sheet_links = []
    total_size = 0  
    total_size_csv = 0
    total_rows = 0
    dataframes = []

    for i, demonstrativo_cvm in enumerate(demonstrativos_cvm):
        print(remaining_time(start_time, len(demonstrativos_cvm), i))
        # Retrieve the list of files based on the specified 'demonstrativo_cvm'
        download_files = [file_link for file_link in files_list if 'meta' not in file_link and demonstrativo_cvm in file_link]

        # Iterate through the list of URLs
        start_time_2 = time.time()
        for j, zip_url in enumerate(download_files):
            print('  ' + remaining_time(start_time_2, len(download_files), j))
            # Download the zip file
            response = requests.get(zip_url)
            
            # Check if the download was successful
            if response.status_code == 200:
                # Get the size of the downloaded file
                file_size = len(response.content)/(1024 ** 2)
                total_size += file_size
                print(f'{zip_url}, {file_size:.3f} Mb, {total_size:.3f} Mb total')

                # Extract the zip file in memory
                with zipfile.ZipFile(BytesIO(response.content)) as zip_ref:
                    # Iterate through the files in the zip
                    start_time_3 = time.time()
                    for k, file_info in enumerate(zip_ref.infolist()):
                        # print('  ' + '  ' + remaining_time(start_time_3, len(zip_ref.infolist()), k))
                        # Check if the file is a CSV
                        if file_info.filename.lower().endswith('.csv'):
                            # file size
                            csv_size = file_info.file_size/(1024 ** 2)
                            total_size_csv += csv_size/(1024 ** 1)

                            # Extract the CSV file
                            csv_content = zip_ref.read(file_info.filename)
                            csv_filename = os.path.basename(file_info.filename)

                            # Extract metadata from the CSV filename
                            meta_csv = csv_filename.replace('cia_aberta_', '').replace('.csv', '').split('_')
                            ano = meta_csv[-1]
                            demonstrativo_cvm = meta_csv[0]
                            meta_csv = meta_csv[1:-1]
                            if len(meta_csv) > 0:
                                agrupamento = meta_csv[-1]
                                meta_csv = meta_csv[:-1]
                            else:
                                agrupamento = ''
                            balance_sheet = '_'.join(meta_csv)

                            # Read CSV content into a pandas DataFrame
                            csv_data = pd.read_csv(BytesIO(csv_content), encoding='iso-8859-1', sep=';')

                            # Add metadata columns to the DataFrame
                            csv_data.insert(0, 'FILE_NAME', csv_filename)
                            csv_data.insert(1, 'demonstrativo_cvm', demonstrativo_cvm)
                            csv_data.insert(2, 'BALANCE_SHEET', balance_sheet)
                            csv_data.insert(3, 'ANO', ano)
                            csv_data.insert(4, 'AGRUPAMENTO', agrupamento)

                            # Append the DataFrame to the list
                            dataframes.append(csv_data)
                            total_rows += len(csv_data)
                            # print(f'{file_info.filename} {csv_size:.3f} Mb, {total_size_csv:.3f} Gb total, {total_rows:,.0f} total rows')
    print(f'Total {len(dataframes)} databases found and {total_rows} downloaded')


    return dataframes

In [12]:
# clean text
def clean_text(text):
    if isinstance(text, str):  # Check if the value is a string
        cleaned_text = unidecode.unidecode(text).translate(str.maketrans('', '', string.punctuation)).upper().strip()
        return cleaned_text

In [13]:
# remove words from cell
def clean_cell(cell):
    words_to_remove = ['  EM LIQUIDACAO', ' EM LIQUIDACAO', ' EXTRAJUDICIAL', '  EM RECUPERACAO JUDICIAL', '  EM REC JUDICIAL', ' EM RECUPERACAO JUDICIAL']
    for word in words_to_remove:
        if word in cell:
            cell = cell.replace(word, '').strip()
    return cell



In [14]:
def clean_DT_INI_EXERC(demo_cvm):
    print('double clean dataframes')
    try:
        lines_removed = 0
        for i, (year, df) in enumerate(demo_cvm.items()):
            size = len(df)
            
            # Apply the condition to filter the DataFrame
            mask = (df['DT_INI_EXERC'] == pd.to_datetime(str(year) + '-01-01')) | df['DT_INI_EXERC'].isna()
            df_filtered = df[mask].copy()  # Make a copy to avoid modifying the original DataFrame
            
            # Update the 'MATH_MAGIC' column for the filtered rows using .loc indexer
            df_filtered.loc[:, 'MATH_MAGIC'] = False
            
            # Update the dictionary with the filtered DataFrame
            demo_cvm[year] = df_filtered
            
            lines_removed += size - len(df_filtered)
            print(year, remaining_time(start_time, len(demo_cvm), i))
        print(f'{lines_removed} lines removed')
    except Exception as e:
        pass
    return demo_cvm


In [15]:
# clean dataframes
def clean_dataframes(dict_of_df):
    print('clean dataframes')
    col_datetime = ['DT_REFER', 'DT_FIM_EXERC', 'DT_INI_EXERC']

    for i, (year, df) in enumerate(dict_of_df.items()):
        print(year, remaining_time(start_time, len(dict_of_df), i))
        # remove extra rows
        try:
            df = df[df['ORDEM_EXERC'] == 'ÚLTIMO']
            df = df.drop(columns=['ORDEM_EXERC'])
        except Exception as e:
            # print(e)
            pass

        # # remove extra rows
        # try:
        #     df = df[(df['DT_INI_EXERC'] == pd.to_datetime(str(year) + '-01-01')) | df['DT_INI_EXERC'].isna()]
        # except Exception as e:
        #     pass
        
        # Clean up Text
        try:
            df['DENOM_CIA'] = df['DENOM_CIA'].apply(clean_text)
        except Exception as e:
            pass

        # to datetime
        try:
            df[col_datetime] = df[col_datetime].apply(pd.to_datetime)
        except:
            pass


        # remove specific words
        df['DENOM_CIA'] = df['DENOM_CIA'].apply(clean_cell)

        dict_of_df[year] = df

    return dict_of_df


In [16]:
# make yearly dict_of_df
def yearly(df_list):
    df_y = {}
    print('group by year')
    start_time = time.time()

    # Iterate through each DataFrame in the 'demo_cvm' list
    for i, df in enumerate(df_list):
        # Get the year from the 'ANO' column
        # year = int(df['ANO'].iloc[0])  # Assuming the 'ANO' value is the same for all rows in a DataFrame
        year = pd.to_datetime(df['DT_REFER']).dt.year.iloc[0]  # Extracting the year from the date
        print(year, remaining_time(start_time, len(df_list), i))

        # Check if the year is already a key in the dictionary, if not, create a list for it
        if year not in df_y:
            df_y[year] = []
        
        # Append the DataFrame to the list for the respective year
        df_y[year].append(df)

    print('concatenating')
    start_time = time.time()
    for i, (year, df_list) in enumerate(df_y.items()):
        print(year, remaining_time(start_time, len(df_y), i))
        df_y[year] = pd.concat(df_list, ignore_index=True)

    return df_y

In [17]:
# group_by_year dict
def group_by_year(dataframes):
    demo_cvm = [df for df in dataframes if len(df) > 0 and ('con' in df['FILE_NAME'][0] or 'ind' in df['FILE_NAME'][0])]
    links = [df for df in dataframes if len(df) > 0 and ('con' not in df['FILE_NAME'][0] and 'ind' not in df['FILE_NAME'][0])]

    demo_cvm = yearly(demo_cvm)
    links = yearly(links)

    # print('clean up dataframes')
    demo_cvm = clean_dataframes(demo_cvm)
    links = clean_dataframes(links)

    # Rename column for consistency
    for year in links.keys():
        links[year].rename(columns={'VERSAO': 'VERSAO_LINK'}, inplace=True)
    
    return demo_cvm, links

In [18]:
# função auxiliar para download pdf
def download_pdf(df, url):
    # Base directory to save PDFs
    output_dir = 'assets/pdf'
    os.makedirs(output_dir, exist_ok=True)

    total_size = 0  # Initialize cumulative total size

    for i, row in df.iterrows():
        response = requests.get(url.format(ID_DOC=row['ID_DOC']))
        if response.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
                file_size = len(response.content)/(1024 ** 2)
                total_size += file_size
                for file_info in zip_ref.infolist():
                    if file_info.filename.lower().endswith('.pdf'):
                        pdf_content = zip_ref.read(file_info.filename)
                        filename = f"{clean_text(row['DENOM_CIA'])} {row['DT_REFER']} VERSAO_{row['VERSAO']} {row['ID_DOC']}.pdf"
                        filepath = os.path.join(output_dir, filename)
                        with open(filepath, 'wb') as pdf_file:
                            pdf_file.write(pdf_content)
                        print(f'{i+1}/{len(df)}, {filename}, {file_size:.3f} Mb, {total_size:.3f} Mb total')
    return df

In [19]:
# get list of companies by AGRUPAMENTO ['ind', 'con']
def get_companies_by_str_port(df):
    # str_port = structured report = relatório estruturado
    
    # Create a pivot table to count the occurrences of 'ind' and 'con' for each 'DENOM_CIA' and 'DT_REFER'
    pivot_table = df.pivot_table(index=['DENOM_CIA', 'DT_REFER'], columns='AGRUPAMENTO', aggfunc='size', fill_value=0)

    # Apply a conditional mapping to convert counts to 1 if count > 0, and 0 otherwise
    pivot_table = pivot_table.applymap(lambda x: True if x > 0 else False)
    pivot_table = pivot_table[['ind'] + [col for col in pivot_table.columns if col != 'ind' and col != 'con'] + ['con']]

    # Get the unique combinations of rows as tuples
    combinations = set(map(tuple, pivot_table.to_numpy()))

    # Create a dictionary to store the combinations of 'con' and 'ind' as keys and corresponding 'DENOM_CIA' as values
    companies_by_str_port  = {}

    # Iterate through the unique combinations and find matching 'DENOM_CIA'
    for combination in combinations:
        relest_individual = combination[0]
        relest_consolidado = combination[1]
        cias = pivot_table[(pivot_table['ind'] == relest_individual) & (pivot_table['con'] == relest_consolidado)].index.get_level_values('DENOM_CIA').unique()
        key = ('ind', 'con')
        if relest_consolidado and not relest_individual:
            key = 'con'
        if not relest_consolidado and relest_individual:
            key = 'ind'

        companies_by_str_port [key] = cias

    return companies_by_str_port

## Content

### Test API and macrodata

In [ ]:
!pip install wbdata

In [ ]:
import wbdata

In [ ]:
df = pd.DataFrame(wbdata.get_data('NY.GDP.MKTP.CD'))
df.head(3)

In [ ]:
import requests
from bs4 import BeautifulSoup

URL = 'https://www.tesourodireto.com.br/titulos/precos-e-taxas.htm'
response = requests.get(URL)

soup = BeautifulSoup(response.content, 'html.parser')

# Buscando os títulos e rendimentos
titulos = soup.find_all('td', class_='listing__col--title')
rendimentos = soup.find_all('td', class_='listing__col--rate')

for titulo, rendimento in zip(titulos, rendimentos):
    print(titulo.text.strip(), "-", rendimento.text.strip())


In [ ]:
# URL of the CSV file
url = 'https://www.tesourotransparente.gov.br/ckan/dataset/df56aa42-484a-4a59-8184-7676580c81e3/resource/796d2059-14e9-44e3-80c9-2d9e30b405c1/download/PrecoTaxaTesouroDireto.csv'

# Read the CSV file with the specified separator and decimal point
td_hist = pd.read_csv(url, sep=';', decimal=',')
td_hist.head(3)

In [ ]:
td_groups = td_hist.groupby(['Tipo Titulo', 'Data Vencimento'])
for group, df in td_groups:
    print(group)
    # df[['Data Base', 'Taxa Compra Manha']].set_index('Data Base').plot()
len(td_groups)

In [ ]:
distritos = pd.read_json('https://servicodados.ibge.gov.br/api/v1/localidades/distritos')
pesquisas = pd.read_json('http://servicodados.ibge.gov.br/api/v2/metadados/pesquisas')

In [ ]:
distritos

### df_pre x df: remover older, update  


In [20]:
last_quarters = ['3', '4']
all_quarters = ['6', '7']
start_time = time.time()
app_folder = 'datasets/'

In [22]:
def clean_dataframe(dict_of_df):
    """
    Cleans and preprocesses DataFrames in a dictionary.

    This function takes a dictionary of DataFrames and performs various cleaning operations on each DataFrame.
    It removes extra rows, cleans up text columns, converts specified columns to datetime format,
    and applies a text cleaning function to specific columns.

    Args:
        dict_of_df (dict): A dictionary where keys are years and values are DataFrames.

    Returns:
        dict: A dictionary with cleaned and preprocessed DataFrames.
    """
    # Change data types for columns
    category_columns = ['FILENAME', 'DEMONSTRATIVO', 'BALANCE_SHEET', 'ANO', 'AGRUPAMENTO', 'CNPJ_CIA', 'VERSAO', 'DENOM_CIA', 'CD_CVM', 'GRUPO_DFP', 'MOEDA', 'ESCALA_MOEDA', 'CD_CONTA', 'DS_CONTA','ST_CONTA_FIXA', 'COLUNA_DF', ]
    datetime_columns = ['DT_REFER', 'DT_FIM_EXERC', 'DT_INI_EXERC', ]
    numeric_columns = ['VL_CONTA', ]

    print('... cleaning database')
    start_time = time.time()
    for i, (year, df) in enumerate(dict_of_df.items()):
        print(year, remaining_time(start_time, len(dict_of_df), i))
        
        # Remove extra rows based on specific conditions
        try:
            df = df[df['ORDEM_EXERC'] == 'ÚLTIMO']
            df = df.drop(columns=['ORDEM_EXERC'])
        except Exception as e:
            # print(e)
            pass
        
        # Apply the condition to filter the DataFrame
        try:
            df['DT_INI_EXERC'] = pd.to_datetime(df['DT_INI_EXERC'], errors='coerce')
            mask = (df['DT_INI_EXERC'].dt.month == 1) | (df['DT_INI_EXERC'].isna())
            df = df[mask].copy()  # Make a copy to avoid modifying the original DataFrame
            df = df.drop(columns=['DT_INI_EXERC'])
        except Exception as e:
            pass

        # Clean up text in 'DENOM_CIA' column
        try:
            df['DENOM_CIA'] = df['DENOM_CIA'].apply(clean_text)
        except Exception as e:
            pass

        # Convert specified columns to specified formats
        for column in df.columns:
            if column in category_columns:
                try:
                    df[column] = df[column].astype('category')
                except Exception as e:
                    pass
            elif column in datetime_columns:
                try:
                    df[column] = pd.to_datetime(df[column])
                except Exception as e:
                    pass
            elif column in numeric_columns:
                try:
                    df[column] = pd.to_numeric(df[column], errors='ignore')
                except Exception as e:
                    pass

        dict_of_df[year] = df

    return dict_of_df


In [101]:
def perform_math_magic(demo_cvm, max_iterations=20000000):
    """
    Perform 'magic' calculations on the DataFrame demo_cvm based on specified quarters.

    Args:
        demo_cvm (dict): Dictionary of DataFrames containing financial data.
        last_quarters (list): List of quarters considered as last quarters.
        all_quarters (list): List of quarters considered as all quarters.
        max_iterations (int): Maximum number of iterations to perform.

    Returns:
        dict: Updated demo_cvm with 'magic' calculations.

    This function iterates through the provided demo_cvm DataFrames, performs calculations based on specified quarters,
    and updates the 'VL_CONTA' values where necessary.
    """
    try:
        print('entering the smart mathmagic world... It takes long time, came back tomorrow... ')
        start_time = time.time()
        # Iterate through each year's DataFrame
        for n1, (year, demonstrativo_cvm) in enumerate(demo_cvm.items()):
            companies_by_str_port = get_companies_by_str_port(demonstrativo_cvm)
            print(f"{year} {len(demonstrativo_cvm):,.0f} lines, {len(demonstrativo_cvm['DENOM_CIA'].unique())} companies, {'/'.join([f'{len(companies)} {key}' for key, companies in companies_by_str_port.items()])}")
            print(year, remaining_time(start_time, len(demo_cvm), n1))
            # Convert DT_REFER to datetime
            demonstrativo_cvm['DT_REFER'] = pd.to_datetime(demonstrativo_cvm['DT_REFER'])
            groups = demonstrativo_cvm.groupby(['DENOM_CIA', 'AGRUPAMENTO'], group_keys=False)
            start_time_2 = time.time()
            for n2, (key, group) in enumerate(groups):
            #   if key[0] == 'ALPARGATAS SA':
                print('  ', remaining_time(start_time_2, len(groups), n2))
                company = key[0]
                agg = key[1]
                subgroups = group.groupby(['CD_CONTA', 'DS_CONTA'], group_keys=False)
                
                start_time_3 = time.time()
                for n3, (index, df) in enumerate(subgroups):
                    # print('  ', '  ', remaining_time(start_time_3, len(subgroups), n3))
                    conta_first = index[0][0]
                   
                    try:
                        i1 = df[df['DT_REFER'].dt.quarter == 1].index[0]
                        q1 = df[df['DT_REFER'].dt.quarter == 1]['VL_CONTA'].iloc[0]
                    except Exception:
                        q1 = 0
                    try:
                        i2 = df[df['DT_REFER'].dt.quarter == 2].index[0]
                        q2 = df[df['DT_REFER'].dt.quarter == 2]['VL_CONTA'].iloc[0]
                    except Exception:
                        q2 = 0
                    try:
                        i3 = df[df['DT_REFER'].dt.quarter == 3].index[0]
                        q3 = df[df['DT_REFER'].dt.quarter == 3]['VL_CONTA'].iloc[0]
                    except Exception:
                        q3 = 0
                    try:
                        i4 = df[df['DT_REFER'].dt.quarter == 4].index[0]
                        q4 = df[df['DT_REFER'].dt.quarter == 4]['VL_CONTA'].iloc[0]
                    except Exception:
                        q4 = 0

                    update = False
                    try:
                        # Perform calculations based on specified quarters and update flag
                        if conta_first in last_quarters and i4:
                            q4 = q4 - (q3)
                            update = True
                        elif conta_first in all_quarters and i2 and i3 and i4:
                            q4 = q4 - (q3)
                            q3 = q3 - (q2)
                            q2 = q2 - (q1)
                            update = True
                    except Exception as e:
                        update = False

                    if update:
                        # Update 'VL_CONTA' values
                        try:
                           demonstrativo_cvm.loc[i1, 'VL_CONTA'] = q1
                        except Exception as e:
                            pass
                        try:
                           demonstrativo_cvm.loc[i2, 'VL_CONTA'] = q2
                        except Exception as e:
                            pass
                        try:
                           demonstrativo_cvm.loc[i3, 'VL_CONTA'] = q3
                        except Exception as e:
                            pass
                        try:
                           demonstrativo_cvm.loc[i4, 'VL_CONTA'] = q4
                        except Exception as e:
                            pass

                    if n3 > max_iterations:
                        break
                if n2 > max_iterations:
                    break
            demo_cvm[year] = demonstrativo_cvm
            if n1 > max_iterations:
                break
    except Exception as e:
       pass
    return demo_cvm


In [251]:
cvm_now = load_pkl(f'{app_folder}cvm_now')
cvm_new = load_pkl(f'{app_folder}cvm_new')


In [266]:
# Getting all unique years from both dictionaries
all_keys = set(cvm_now.keys()).union(cvm_new.keys())
cols =        ['DEMONSTRATIVO', 'BALANCE_SHEET', 'ANO', 'AGRUPAMENTO', 'CNPJ_CIA', 'DT_REFER', 'DENOM_CIA', 'DT_INI_EXERC', 'DT_FIM_EXERC', 'CD_CONTA', 'DS_CONTA', 'VL_CONTA']

key_columns = ['FILENAME', 'DEMONSTRATIVO', 'BALANCE_SHEET', 'ANO', 'AGRUPAMENTO', 'CNPJ_CIA', 'DT_REFER', 'VERSAO', 'DENOM_CIA', 'CD_CVM', 'GRUPO_DFP', 'MOEDA', 'ESCALA_MOEDA', 'DT_INI_EXERC', 'DT_FIM_EXERC', 'CD_CONTA', 'DS_CONTA', 'ST_CONTA_FIXA', 'COLUNA_DF']

cvm_merged = {}
cvm_all_year_rows = {}

for year in all_keys:
    try:
        df_columns = cvm_now[year].columns
        half = int(len(cvm_now[year])/2)
    except Exception as e:
        df_columns = cvm_new[year].columns
        half = int(len(cvm_new[year])/2)

    df1 = cvm_now.get(year, pd.DataFrame(columns=df_columns))
    df2 = cvm_new.get(year, pd.DataFrame(columns=df_columns))
    
    # for debug purposes
    df1 = df2[:-100].copy()
    df1.loc[:,'VL_CONTA'] = df1.loc[:,'VL_CONTA']/2
    mask = (df2['DENOM_CIA'] == 'ALPARGATAS SA') & \
        (df2['AGRUPAMENTO'] == 'con') & \
        (df2['CD_CONTA'].isin(['1.01', '3.01', '6.01']))
    df2 = df2[mask]
    
    # Check if df1 doesn't exist and df2 exists
    if df1.shape[0] == 0 and df2.shape[0] != 0:
        cvm_merged[year] = df2

    # Check if df2 doesn't exist and df1 exists
    elif df2.shape[0] == 0 and df1.shape[0] != 0:
        cvm_merged[year] = df1

    # Both df1 and df2 exist (because at least one key exists)
    else:
        df_merged = pd.merge(df1, df2, on=key_columns, how='outer', indicator=True, suffixes=('_now', '_new'))
        
        # Create df_diff to only contain rows from df2
        diff_rows = df_merged[df_merged['_merge'].isin(['right_only', 'both'])].copy()

        # Conditional update of the 'VL_CONTA_new' column based on the '_merge' column value
        df_merged['VL_CONTA_new'] = np.where(df_merged['_merge'] == 'left_only', df_merged['VL_CONTA_now'], df_merged['VL_CONTA_new'])

        # Drop the 'VL_CONTA_now' and '_merge' columns
        df_merged.drop(columns=['VL_CONTA_now', '_merge'], inplace=True)
        
        # Rename the 'VL_CONTA_new' column to 'VL_CONTA'
        df_merged.rename(columns={'VL_CONTA_new': 'VL_CONTA'}, inplace=True)
        df_merged = df_merged[df_columns]

        # for index, row in diff_rows.iterrows():
        #     refer_year = pd.to_datetime(row['DT_REFER']).year  # Extract the year from DT_REFER
        #     cvm_all_year_rows[refer_year] = df_merged[df_merged['DT_REFER'].dt.year == refer_year]


        cvm_merged[year] = df_merged
    break

In [340]:
def adjust_quarters(group_df):
    # Filtering to get VL_CONTA values for each quarter
    q1_value = group_df.loc[group_df['DT_REFER'].dt.month == 3, 'VL_CONTA'].values[0] if not group_df.loc[group_df['DT_REFER'].dt.month == 3, 'VL_CONTA'].empty else 0
    q2_value = group_df.loc[group_df['DT_REFER'].dt.month == 6, 'VL_CONTA'].values[0] if not group_df.loc[group_df['DT_REFER'].dt.month == 6, 'VL_CONTA'].empty else 0
    q3_value = group_df.loc[group_df['DT_REFER'].dt.month == 9, 'VL_CONTA'].values[0] if not group_df.loc[group_df['DT_REFER'].dt.month == 9, 'VL_CONTA'].empty else 0
    q4_value = group_df.loc[group_df['DT_REFER'].dt.month == 12, 'VL_CONTA'].values[0] if not group_df.loc[group_df['DT_REFER'].dt.month == 12, 'VL_CONTA'].empty else 0

    # Adjusting VL_CONTA values as per the formulas
    q4_adj = q4_value - (q3_value + q2_value + q1_value)
    q3_adj = q3_value - (q2_value + q1_value)
    q2_adj = q2_value - q1_value

    # Assigning the adjusted values back to the DataFrame
    group_df.loc[group_df['DT_REFER'].dt.month == 12, 'VL_CONTA'] = q4_adj
    group_df.loc[group_df['DT_REFER'].dt.month == 9, 'VL_CONTA'] = q3_adj
    group_df.loc[group_df['DT_REFER'].dt.month == 6, 'VL_CONTA'] = q2_adj

    return group_df

def filter_last_quarter(group_df):
    # For Q2 and Q3, only keep rows where DT_REFER and DT_INI_EXERC quarters match
    mask = ~group_df['DT_REFER'].dt.quarter.isin([2, 3]) | (group_df['DT_REFER'].dt.quarter == group_df['DT_INI_EXERC'].dt.quarter)
    group_df = group_df[mask]
    return group_df

def adjust_last_quarter(group_df):
    # Adjust Q4 values
    q1_value = group_df[group_df['DT_REFER'].dt.quarter == 1]['VL_CONTA'].sum()
    q2_value = group_df[group_df['DT_REFER'].dt.quarter == 2]['VL_CONTA'].sum()
    q3_value = group_df[group_df['DT_REFER'].dt.quarter == 3]['VL_CONTA'].sum()
    
    mask = group_df['DT_REFER'].dt.quarter == 4
    group_df.loc[mask, 'VL_CONTA'] -= (q1_value + q2_value + q3_value)
    return group_df

In [341]:
# Initialize an empty list to store all the filtered rows
all_filtered_rows = []

for index, row in diff_rows.iterrows():
    # Initialize a filter mask
    mask = (df_merged['DENOM_CIA'] == row['DENOM_CIA']) & \
           (df_merged['AGRUPAMENTO'] == row['AGRUPAMENTO']) & \
           (df_merged['CD_CONTA'] == row['CD_CONTA']) & \
           (df_merged['DS_CONTA'] == row['DS_CONTA'])

    # Get the filtered rows using the mask
    filtered_rows = df_merged[mask].copy()

    # Check if the BALANCE_SHEET value is one of the 'all_quarters'
    if row['BALANCE_SHEET'] in all_quarters:
        adjusted_rows = adjust_quarters(filtered_rows)
        all_filtered_rows.append(adjusted_rows)
    elif row['BALANCE_SHEET'] in last_quarter:
        # Filter and adjust for last quarter
        filtered_rows = filter_last_quarter(filtered_rows)
        adjusted_rows = adjust_last_quarter(filtered_rows)
        all_filtered_rows.append(adjusted_rows)
    else:
        all_filtered_rows.append(filtered_rows)

# Concatenate all the rows (adjusted and non-adjusted) into a single DataFrame
math = pd.concat(all_filtered_rows, axis=0).drop_duplicates()


In [342]:
math

,FILENAME,DEMONSTRATIVO,BALANCE_SHEET,ANO,AGRUPAMENTO,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA,DT_INI_EXERC,COLUNA_DF
49328,itr_cia_aberta_BPA_con_2016.csv,itr,BPA,2016,con,61.079.117/0001-05,2016-03-31,1,ALPARGATAS SA,10456,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,2016-03-31,1.01,Ativo Circulante,2150935.0,S,NaT,NaN
49395,itr_cia_aberta_BPA_con_2016.csv,itr,BPA,2016,con,61.079.117/0001-05,2016-06-30,2,ALPARGATAS SA,10456,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,2016-06-30,1.01,Ativo Circulante,2260506.0,S,NaT,NaN
49462,itr_cia_aberta_BPA_con_2016.csv,itr,BPA,2016,con,61.079.117/0001-05,2016-09-30,1,ALPARGATAS SA,10456,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,2016-09-30,1.01,Ativo Circulante,2179017.0,S,NaT,NaN
1431027,dfp_cia_aberta_BPA_con_2016.csv,dfp,BPA,2016,con,61.079.117/0001-05,2016-12-31,2,ALPARGATAS SA,10456,DF Consolidado - Balanço Patrimonial Ativo,REAL,MIL,2016-12-31,1.01,Ativo Circulante,2262005.0,S,NaT,NaN
521593,itr_cia_aberta_DFC_MI_con_2016.csv,itr,DFC_MI,2016,con,61.079.117/0001-05,2016-03-31,1,ALPARGATAS SA,10456,DF Consolidado - Demonstração do Fluxo de Caixa (Método Indireto),REAL,MIL,2016-03-31,6.01,Caixa Líquido Atividades Operacionais,-8520.0,S,2016-01-01,NaN
521634,itr_cia_aberta_DFC_MI_con_2016.csv,itr,DFC_MI,2016,con,61.079.117/0001-05,2016-06-30,2,ALPARGATAS SA,10456,DF Consolidado - Demonstração do Fluxo de Caixa (Método Indireto),REAL,MIL,2016-06-30,6.01,Caixa Líquido Atividades Operacionais,98487.0,S,2016-01-01,NaN
521675,itr_cia_aberta_DFC_MI_con_2016.csv,itr,DFC_MI,2016,con,61.079.117/0001-05,2016-09-30,1,ALPARGATAS SA,10456,DF Consolidado - Demonstração do Fluxo de Caixa (Método Indireto),REAL,MIL,2016-09-30,6.01,Caixa Líquido Atividades Operacionais,95266.0,S,2016-01-01,NaN
1594714,dfp_cia_aberta_DFC_MI_con_2016.csv,dfp,DFC_MI,2016,con,61.079.117/0001-05,2016-12-31,2,ALPARGATAS SA,10456,DF Consolidado - Demonstração do Fluxo de Caixa (Método Indireto),REAL,MIL,2016-12-31,6.01,Caixa Líquido Atividades Operacionais,82811.0,S,2016-01-01,NaN
1186273,itr_cia_aberta_DRE_con_2016.csv,itr,DRE,2016,con,61.079.117/0001-05,2016-03-31,1,ALPARGATAS SA,10456,DF Consolidado - Demonstração do Resultado,REAL,MIL,2016-03-31,3.01,Receita de Venda de Bens e/ou Serviços,1011568.0,S,2016-01-01,NaN
1186310,itr_cia_aberta_DRE_con_2016.csv,itr,DRE,2016,con,61.079.117/0001-05,2016-06-30,2,ALPARGATAS SA,10456,DF Consolidado - Demonstração do Resultado,REAL,MIL,2016-06-30,3.01,Receita de Venda de Bens e/ou Serviços,1011899.0,S,2016-04-01,NaN


In [53]:
cia_columns = ['DENOM_CIA']
quarter_columns = ['DT_REFER']
agg_columns = ['AGRUPAMENTO']
sheet_columns = ['BALANCE_SHEET']
content_cols = ['CD_CONTA', 'DS_CONTA', 'VL_CONTA']
unique_columns = cia_columns + quarter_columns + agg_columns


In [54]:
company = 'ALPARGATAS SA'
agg = 'con'

filtered_dfs = {}

for year, df in df_old.items():
    # Use boolean indexing to filter by 'DENOM_CIA' and 'AGG_COLUMN' (replace 'AGG_COLUMN' with the actual column name)
    filtered_df = df[(df['DENOM_CIA'] == company) & (df['AGRUPAMENTO'] == agg)]
    
    # Store the filtered DataFrame in a new dictionary
    filtered_dfs[year] = filtered_df
df_old = filtered_dfs

filtered_dfs = {}

for year, df in df_new.items():
    # Use boolean indexing to filter by 'DENOM_CIA' and 'AGG_COLUMN' (replace 'AGG_COLUMN' with the actual column name)
    filtered_df = df[(df['DENOM_CIA'] == company) & (df['AGRUPAMENTO'] == agg)]
    
    # Store the filtered DataFrame in a new dictionary
    filtered_dfs[year] = filtered_df
df_new = filtered_dfs

In [66]:
df_diff = {}
df_math = {}
for year in df_new.keys():
    df1 = df_old[year]
    df2 = df_new[year]
    if df1.shape == df2.shape:
        mask_diff = df1['VL_CONTA'] != df2['VL_CONTA']
        df_diff[year] = df_new[year][mask_diff]
        for i, (idx, row) in enumerate(df_diff[year].iterrows()):
            # Print information about the difference
            print(
                f"{row['DENOM_CIA']}, {row['AGRUPAMENTO']}, {row['DT_REFER'].strftime('%Y-%m-%d')}, "
                f"Old Value: {int(df1.loc[idx, 'VL_CONTA'])} -> New Value: {int(row['VL_CONTA'])}, "
                f"{row['CD_CONTA']}, {row['DS_CONTA']}")

            # Create filters to identify rows in df2 that match the current difference
            filter_mask_cia = df2['DENOM_CIA'] == row['DENOM_CIA']
            filter_mask_agg = df2['AGRUPAMENTO'] == row['AGRUPAMENTO']
            filter_mask_conta = df2['CD_CONTA'] == row['CD_CONTA']
            filter_mask_year = df2['DT_REFER'].dt.year == row['DT_REFER'].year

            # Combine filters to create a mask for the matching rows in df2
            mask = filter_mask_cia & filter_mask_agg & filter_mask_conta & filter_mask_year

            # Append the filtered matching rows to the list
            df_math[year].append(df2[mask])



In [24]:
cia_columns = ['DENOM_CIA']
quarter_columns = ['DT_REFER']
agg_columns = ['AGRUPAMENTO']
sheet_columns = ['BALANCE_SHEET']
content_cols = ['CD_CONTA', 'DS_CONTA', 'VL_CONTA']
unique_columns = cia_columns + quarter_columns + agg_columns


In [25]:
mask_cia = df_new[2014]['DENOM_CIA'] == 'ALPARGATAS SA'
mask_agg = df_new[2014]['AGRUPAMENTO'] == 'con'
mask_quarter = df_new[2014]['DT_REFER'] == '2014-06-30'
mask_sheet = df_new[2014]['BALANCE_SHEET'] == 'BPA'
mask_CD_CONTA = df_new[2014]['CD_CONTA'] == '1'
mask = mask_cia & mask_agg & mask_quarter & mask_sheet & mask_CD_CONTA
df_new[2014].loc[mask, 'VL_CONTA'] = 1000000.0


mask_cia = df_new[2015]['DENOM_CIA'] == 'ALPARGATAS SA'
mask_agg = df_new[2015]['AGRUPAMENTO'] == 'con'
mask_quarter = df_new[2015]['DT_REFER'] == '2015-09-30'
mask_sheet = df_new[2015]['BALANCE_SHEET'] == 'DRE'
mask_CD_CONTA = df_new[2015]['CD_CONTA'] == '3.01'
mask = mask_cia & mask_agg & mask_quarter & mask_sheet & mask_CD_CONTA
df_new[2015].loc[mask, 'VL_CONTA'] = 1000000.0


mask_cia = df_new[2016]['DENOM_CIA'] == 'ALPARGATAS SA'
mask_agg = df_new[2016]['AGRUPAMENTO'] == 'con'
mask_quarter = df_new[2016]['DT_REFER'] == '2016-12-31'
mask_sheet = df_new[2016]['BALANCE_SHEET'] == 'DFC_MI'
mask_CD_CONTA = df_new[2016]['CD_CONTA'] == '6.01'
mask = mask_cia & mask_agg & mask_quarter & mask_sheet & mask_CD_CONTA
df_new[2016].loc[mask, 'VL_CONTA'] = 1000000.0


In [ ]:
# separa até o ano anterior ao atual e atualiza só a diferença
# para o ano atual, faz todo o mathmagic

In [51]:
def get_diff(df1, df2):
    # Create a mask to identify rows in df2 where 'VL_CONTA' is different from df1
    mask_diff = df1['VL_CONTA'] != df2['VL_CONTA']

    # Initialize an empty list to store DataFrames with differences
    df_diff = []

    # Iterate through rows in df2 that have differences in 'VL_CONTA'
    for i, (idx, row) in enumerate(df2[mask_diff].iterrows()):
        # Print information about the difference
        print(
            f"{row['DENOM_CIA']}, {row['AGRUPAMENTO']}, {row['DT_REFER'].strftime('%Y-%m-%d')}, "
            f"Old Value: {int(df1.loc[idx, 'VL_CONTA'])} -> New Value: {int(row['VL_CONTA'])}, "
            f"{row['CD_CONTA']}, {row['DS_CONTA']}")

        # Create filters to identify rows in df2 that match the current difference
        filter_mask_cia = df2['DENOM_CIA'] == row['DENOM_CIA']
        filter_mask_agg = df2['AGRUPAMENTO'] == row['AGRUPAMENTO']
        filter_mask_conta = df2['CD_CONTA'] == row['CD_CONTA']
        filter_mask_year = df2['DT_REFER'].dt.year == row['DT_REFER'].year

        # Combine filters to create a mask for the matching rows in df2
        mask = filter_mask_cia & filter_mask_agg & filter_mask_conta & filter_mask_year

        # Append the filtered matching rows to the list
        df_diff.append(df2[mask])

    # If there are matching rows, concatenate them into a single DataFrame
    if df_diff:
        df_diff = pd.concat(df_diff, ignore_index=False)

    # Return the DataFrame containing differences
    return df_diff


In [108]:
def update_df_math(df_old, df_new):
    df_math = {}  # Initialize an empty dictionary to store DataFrames with differences

    # Iterate through the years and DataFrames in df_new
    for year, df in df_new.items():
        # Check if the corresponding DataFrame exists in df_old and if the year is in the past
        if year in df_old and df_old[year].shape == df.shape and year < datetime.datetime.now().year:
            
            # Call the get_diff function to identify and extract differences
            dfs = get_diff(df_old[year], df_new[year])
            
            # Check if there are differences found
            if len(dfs) > 0:
                # Store the differences in the dictionary using the year as the key
                df_math[year] = dfs
    # # Add the current year's DataFrame from df_new to df_math_y
    current_year = datetime.datetime.now().year
    if current_year in df_new:
        df_math[current_year] = df_new[current_year]

    return df_math


ALPARGATAS SA, con, 2014-06-30, Old Value: 3320525 -> New Value: 1000000, 1, Ativo Total
ALPARGATAS SA, con, 2015-09-30, Old Value: 3092633 -> New Value: 1000000, 3.01, Receita de Venda de Bens e/ou Serviços
ALPARGATAS SA, con, 2016-12-31, Old Value: 340971 -> New Value: 1000000, 6.01, Caixa Líquido Atividades Operacionais


In [111]:
def merge_cvm_math(demo_cvm, df_math):
    for year in demo_cvm.keys():
        # Update df_cvm['VL_CONTA'] using the values from df_math['VL_CONTA']
        demo_cvm[year].loc[df_math[year].index, 'VL_CONTA'] = df_math[year]['VL_CONTA']
    return demo_cvm


In [110]:
df_math = update_df_math(df_old, df_new)
df_math = perform_math_magic(df_math)
dre_cvm = merge_cvm_math(demo_cvm, df_math)

entering the smart mathmagic world... It takes long time, came back tomorrow... 
2014 4 lines, 1 companies, 1 con/682 ('ind', 'con')
2014 25.00% 1+3, 0.034123s per item, Remaining: 0h 00m 00s
   100.00% 1+0, 0.003118s per item, Remaining: 0h 00m 00s
2015 4 lines, 1 companies, 1 con/664 ('ind', 'con')
2015 50.00% 2+2, 0.037802s per item, Remaining: 0h 00m 00s
   100.00% 1+0, 0.003083s per item, Remaining: 0h 00m 00s
2016 4 lines, 1 companies, 1 con/654 ('ind', 'con')
2016 75.00% 3+1, 0.038755s per item, Remaining: 0h 00m 00s
   100.00% 1+0, 0.003026s per item, Remaining: 0h 00m 00s
2023 1,110,501 lines, 704 companies, 254 ind/4 con/704 ('ind', 'con')
2023 100.00% 4+0, 0.067627s per item, Remaining: 0h 00m 00s
   0.09% 1+1151, 2.050958s per item, Remaining: 0h 39m 20s
   0.17% 2+1150, 4.363199s per item, Remaining: 1h 23m 37s
   0.26% 3+1149, 4.845485s per item, Remaining: 1h 32m 47s
   0.35% 4+1148, 4.880047s per item, Remaining: 1h 33m 22s
   0.43% 5+1147, 5.626426s per item, Remaining

KeyboardInterrupt: 

In [ ]:
df_merged = {}

# Iterating over each year's dataframe in df1
for year, df in df1.items():
    print(year, len(df))
    
    # If the year doesn't exist in df2, simply add the dataframe from df1
    if year not in df2:
        df_merged[year] = df
        continue

    # Merge on the desired columns. The 'how' parameter ensures we keep all rows from df1 for the given year.
    merged_df = df.merge(df2[year], on=['DENOM_CIA', 'AGRUPAMENTO', 'CD_CONTA', 'DT_REFER'], how='left', indicator=True)

    # Rows that are in df1 but not in df2 will have the indicator column set to 'left_only'
    df_filtered = merged_df[merged_df['_merge'] == 'left_only'][df.columns]
    
    # Appending df2's current year dataframe to df_filtered
    df_merged[year] = pd.concat([df_filtered, df2[year]], ignore_index=True)

# Handling years that might be in df2 but not in df1
for year in df2:
    if year not in df1:
        df_merged[year] = df2[year]


In [ ]:
# We'll consider all unique years present in both dictionaries
all_years = set(df1.keys()) | set(df2.keys())

# Iterate over each year
for year in all_years:
    # If the year is present in both df1 and df2
    if year in df1 and year in df2:
        # Merge the two dataframes for the given year on all columns
        # We use an outer merge to retain all rows and use the indicator column to track the origin
        merged_df = df1[year].merge(df2[year], how='outer', indicator=True, on=df1[year].columns.tolist())
        
        # Filter rows that are only in df1 using the indicator column
        unique_df1_rows = merged_df[merged_df['_merge'] == 'left_only'][df1[year].columns.tolist()]
        
        # Update df2[year] by concatenating the unique rows from df1[year]
        df2[year] = pd.concat([df2[year], unique_df1_rows], ignore_index=True)
        
    # If the year is only present in df1 and not in df2, copy the entire df1[year] to df2
    elif year in df1:
        df2[year] = df1[year]

# No need to handle years that are only in df2 since we're updating df2 directly.



In [ ]:
import pandas as pd

# These are the key columns based on which we'll determine if rows are the same between df1 and df2.
key_columns = ['DENOM_CIA', 'AGRUPAMENTO', 'CD_CONTA', 'DT_REFER']

# Iterate over each year in df1
for year in df1:
    
    # Check if the year exists in df2. If not, simply assign df1's dataframe to df2.
    if year not in df2:
        df2[year] = df1[year]
        continue
    
    # Merge the dataframes for the current year based on the key columns.
    # This is a left merge on df1 with respect to df2. So, all rows from df1[year] will be retained.
    # The 'indicator' parameter adds a special column '_merge' to the result which tells us from which dataframe the row came.
    merged_df = df1[year].merge(df2[year], on=key_columns, how='left', indicator=True)
    
    # Filter out rows that are only in df1 (i.e., not in df2).
    # These are the rows we need to append to df2[year].
    df1_only = merged_df[merged_df['_merge'] == 'left_only']
    
    # Drop the '_merge' column as it's not needed in the final output.
    df1_only = df1_only.drop(columns=['_merge'])
    
    # Concatenate the rows from df1 that are not in df2 to df2[year].
    # This updates df2[year] to include rows from both dataframes, but without any duplicates based on our key columns.
    df2[year] = pd.concat([df2[year], df1_only], ignore_index=True)

# At the end of this loop, df2 will have been updated to include rows from both df1 and df2 for each year, 
# without any duplicate rows based on the key columns.


In [ ]:
# The key columns used to determine unique rows
key_columns = ['DENOM_CIA', 'AGRUPAMENTO', 'CD_CONTA', 'DT_REFER']

# Create an empty dictionary to store the merged dataframes for each year
df_merged = {}

# Iterate over each year present in either df1 or df2
for year in set(df1.keys()).union(df2.keys()):
    # Check if the year exists in df2 (the newer dataframe)
    if year in df2:
        # Start with the data from df2 for this year
        merged_df = df2[year].copy()
        
        # Check if the year also exists in df1 (the older dataframe)
        if year in df1:
            # Determine the rows in df1[year] that are not present in df2[year]
            # based on the key columns
            mask = ~df1[year].set_index(key_columns).index.isin(df2[year].set_index(key_columns).index)
            
            # Filter df1[year] to only these rows
            extra_rows = df1[year][mask]
            
            # Append these rows to the merged dataframe
            merged_df = merged_df.append(extra_rows)
    else:
        # If the year doesn't exist in df2, then just use the data from df1
        merged_df = df1[year].copy()

    # Store the merged dataframe for this year in the df_merged dictionary
    df_merged[year] = merged_df

# At this point, df_merged will have the merged data for each year
print(df_merged)


### Download clean and organize databases

In [ ]:
base_cvm = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/'

In [ ]:
companies_pre = load_pkl(f'companies_pre')
companies_pre['ALPARGATAS SA']
df = companies_pre['ALPARGATAS SA']

mask_CD_CONTA = df['CD_CONTA'] == '6.01'
mask_AGRUPAMENTO = df['AGRUPAMENTO'] == 'con'
mask_DT_INI_EXERC = df['DT_INI_EXERC'].dt.month == 1
mask = mask_CD_CONTA & mask_AGRUPAMENTO & mask_DT_INI_EXERC

df[mask][['DT_REFER', 'VL_CONTA']].set_index('DT_REFER')

In [ ]:
companies = load_pkl(f'companies')
companies['ALPARGATAS SA']
df = companies['ALPARGATAS SA']

mask_CD_CONTA = df['CD_CONTA'] == '6.01'
mask_AGRUPAMENTO = df['AGRUPAMENTO'] == 'con'
mask_DT_INI_EXERC = df['DT_INI_EXERC'].dt.month == 1
mask = mask_CD_CONTA & mask_AGRUPAMENTO & mask_DT_INI_EXERC

df[mask][['DT_REFER', 'VL_CONTA']].set_index('DT_REFER').plot()

In [ ]:
demo_cvm[2019].info()

### By Year

In [ ]:
def mathmagic(df):
    df['DT_REFER'] = pd.to_datetime(df['DT_REFER'])
    try:
        i1 = df[df['DT_REFER'].dt.quarter == 1].index[0]
        q1 = df[df['DT_REFER'].dt.quarter == 1]['VL_CONTA'].iloc[0]
    except Exception as e:
        q1 = 0
    try:
        i2 = df[df['DT_REFER'].dt.quarter == 2].index[0]
        q2 = df[df['DT_REFER'].dt.quarter == 2]['VL_CONTA'].iloc[0]
    except Exception as e:
        q2 = 0
    try:
        i3 = df[df['DT_REFER'].dt.quarter == 3].index[0]
        q3 = df[df['DT_REFER'].dt.quarter == 4]['VL_CONTA'].iloc[0]
    except Exception as e:
        q3 = 0
    try:
        i4 = df[df['DT_REFER'].dt.quarter == 4].index[0]
        q4 = df[df['DT_REFER'].dt.quarter == 4]['VL_CONTA'].iloc[0]
    except Exception as e:
        q4 = 0


        # try:
        #     # Perform calculations based on quarters and update flag
        #     if conta_first in last_quarters:
        #         if not demonstrativo_cvm.loc[i4, 'MATH_MAGIC']:
        #             q4 = q4 - (q3 + q2 + q1)
        #         update = True
        #     elif conta_first in all_quarters:
        #         if not demonstrativo_cvm.loc[i2, 'MATH_MAGIC']:
        #             q2 = q2 - (q1)
        #         if not demonstrativo_cvm.loc[i3, 'MATH_MAGIC']:
        #             q3 = q3 - (q2 + q1)
        #         if not demonstrativo_cvm.loc[i4, 'MATH_MAGIC']:
        #             q4 = q4 - (q3 + q2 + q1)
        #         update = True
        # except Exception as e:
        #     update = False

        # # Update demo_cvm data based on calculations
        # if update:
        #     demonstrativo_cvm.loc[i2, ['VL_CONTA', 'MATH_MAGIC']] = [q2, True]
        #     demonstrativo_cvm.loc[i3, ['VL_CONTA', 'MATH_MAGIC']] = [q3, True]
        #     demonstrativo_cvm.loc[i4, ['VL_CONTA', 'MATH_MAGIC']] = [q4, True]

    return df

In [ ]:
for k, v in companies_by_str_port.items():
    print(k)
    print(type(v))

In [ ]:
import time
import pandas as pd

def perform_math_magic(demo_cvm, last_quarters, all_quarters, max_iterations=20000000):
    """
    Perform 'magic' calculations on the DataFrame demo_cvm based on specified quarters.

    Args:
        demo_cvm (dict): Dictionary of DataFrames containing financial data.
        last_quarters (list): List of quarters considered as last quarters.
        all_quarters (list): List of quarters considered as all quarters.
        max_iterations (int): Maximum number of iterations to perform.

    Returns:
        dict: Updated demo_cvm with 'magic' calculations.

    This function iterates through the provided demo_cvm DataFrames, performs calculations based on specified quarters,
    and updates the 'VL_CONTA' values and 'MATH_MAGIC' flag where necessary.
    """
    start_time = time.time()

    # Iterate through each year's DataFrame
    for n1, (year, demonstrativo_cvm) in enumerate(demo_cvm.items()):
        companies_by_str_port = get_companies_by_str_port(demonstrativo_cvm)
        groups = demonstrativo_cvm.groupby(['DENOM_CIA', 'AGRUPAMENTO'], group_keys=False)
        
        start_time_2 = time.time()
        for n2, (key, group) in enumerate(groups):
            company = key[0]
            agg = key[1]
            subgroups = group.groupby(['CD_CONTA', 'DS_CONTA'], group_keys=False)
            
            start_time_3 = time.time()
            for n3, (index, df) in enumerate(subgroups):
                conta_first = index[0][0]
                # Convert DT_REFER to datetime
                df['DT_REFER'] = pd.to_datetime(df['DT_REFER'])
                
                # Create 'MATH_MAGIC' column if not exists
                if 'MATH_MAGIC' not in demonstrativo_cvm.columns:
                    demonstrativo_cvm['MATH_MAGIC'] = False

                try:
                    i1 = df[df['DT_REFER'].dt.quarter == 1].index[0]
                    q1 = df[df['DT_REFER'].dt.quarter == 1]['VL_CONTA'].iloc[0]
                except Exception:
                    q1 = 0
                try:
                    i2 = df[df['DT_REFER'].dt.quarter == 2].index[0]
                    q2 = df[df['DT_REFER'].dt.quarter == 2]['VL_CONTA'].iloc[0]
                except Exception:
                    q2 = 0
                try:
                    i3 = df[df['DT_REFER'].dt.quarter == 3].index[0]
                    q3 = df[df['DT_REFER'].dt.quarter == 4]['VL_CONTA'].iloc[0]
                except Exception:
                    q3 = 0
                try:
                    i4 = df[df['DT_REFER'].dt.quarter == 4].index[0]
                    q4 = df[df['DT_REFER'].dt.quarter == 4]['VL_CONTA'].iloc[0]
                except Exception:
                    q4 = 0

                update = False
                try:
                    # Perform calculations based on specified quarters and update flag
                    if conta_first in last_quarters:
                        if not demonstrativo_cvm.loc[i4, 'MATH_MAGIC']:
                            q4 = q4 - (q3 + q2 + q1)
                        update = True
                    elif conta_first in all_quarters:
                        if not demonstrativo_cvm.loc[i2, 'MATH_MAGIC']:
                            q2 = q2 - (q1)
                        if not demonstrativo_cvm.loc[i3, 'MATH_MAGIC']:
                            q3 = q3 - (q2 + q1)
                        if not demonstrativo_cvm.loc[i4, 'MATH_MAGIC']:
                            q4 = q4 - (q3 + q2 + q1)
                        update = True
                except Exception:
                    update = False

                if update:
                    # Update 'VL_CONTA' values and 'MATH_MAGIC' flag
                    demonstrativo_cvm.loc[i2, ['VL_CONTA', 'MATH_MAGIC']] = [q2, True]
                    demonstrativo_cvm.loc[i3, ['VL_CONTA', 'MATH_MAGIC']] = [q3, True]
                    demonstrativo_cvm.loc[i4, ['VL_CONTA', 'MATH_MAGIC']] = [q4, True]

                if n3 > max_iterations:
                    break
            if n2 > max_iterations:
                break
        if n1 > max_iterations:
            break

    return demo_cvm


In [ ]:
print(i1, i2, i3, i4)

In [ ]:
df

In [ ]:
if conta_first in ['1']:
    print('sim')

In [ ]:
demonstrativo_cvm.loc[i4, 'MATH_MAGIC']

In [ ]:
demonstrativo_cvm.loc[[i1, i2, i3, i4]]['VL_CONTA']

In [ ]:
df['VL_CONTA']

In [ ]:
def by_year(demo_cvm):
    """
    Process demo_cvm data by performing calculations and updates on a per-year basis.

    Args:
        demo_cvm (dict): Dictionary containing demo_cvm data for different years.

    Returns:
        dict: Updated demo_cvm data after processing.

    This function iterates through each year's demo_cvm data, performs calculations,
    and updates the data based on specific conditions. It handles quarter-wise calculations
    and adjustments for various financial data, such as account balances and quarters.

    Args:
        demo_cvm (dict): A dictionary containing demo_cvm data for different years.

    Returns:
        dict: The updated demo_cvm data after applying calculations and adjustments.

    Loop Explanation:
    - The function iterates over each year's demo_cvm data to process the financial data.
    - Inside the year loop, the function groups data by companies and structured reports (ind, con).
    - For each group, it calculates the total number of lines and unique companies involved.
    - It then iterates over subgroups, each representing a company and a specific account.
    - For each subgroup, the function performs calculations based on quarters and updates
      the data accordingly, ensuring correct financial data representation.

    The `n` parameter is used as a limit for iterations during development, providing faster
    testing and debugging. Adjust the value of `n` to match your needs for data processing.

    Example:
        To use this function, pass the `demo_cvm` dictionary containing financial data by year.
        The function will iterate through the years and process financial data accordingly,
        ensuring accurate calculations and adjustments.

    """
    # Define relevant quarters
    last_quarters = ['3', '4']
    all_quarters = ['6', '7']

    # Large number for limiting iterations during development
    n = 1000000000

    # Start time for overall execution
    start_time = time.time()

    # Loop through each year's data in demo_cvm
    for n1, (year, demonstrativo_cvm) in enumerate(demo_cvm.items()):

        # Get companies by structured report (ind, con)
        companies_by_str_port = get_companies_by_str_port(demonstrativo_cvm)
        print(f"{year} {len(demonstrativo_cvm):,.0f} lines, {len(demonstrativo_cvm['DENOM_CIA'].unique())} companies, {'/'.join([f'{len(companies)} {key}' for key, companies in companies_by_str_port.items()])}")
        print(year, remaining_time(start_time, len(demo_cvm), n1))

        # Group by 'DENOM_CIA' and 'AGRUPAMENTO'
        groups = demonstrativo_cvm.groupby(['DENOM_CIA', 'AGRUPAMENTO'], group_keys=False)

        # Start time for grouping by company and account
        start_time_2 = time.time()

        # Iterate through groups (company and account)
        for n2, (key, group) in enumerate(groups):
            print('  ', remaining_time(start_time_2, len(groups), n2))
            company = key[0]
            agg = key[1]
            
            # Group by 'CD_CONTA' and 'DS_CONTA'
            subgroups = group.groupby(['CD_CONTA', 'DS_CONTA'], group_keys=False)

            # Start time for processing subgroups
            start_time_3 = time.time()

            # Iterate through subgroups (account)
            for n3, (index, df) in enumerate(subgroups):
                # print('  ', '  ', remaining_time(start_time_3, len(subgroups), n3))
                conta = index[0]
                conta_first = index[0][0]
                descricao = index[1]

                # Update flag for whether to perform calculations
                update = False

                # Check if all quarters are present in 'DT_REFER' column
                if all(q in df['DT_REFER'].dt.quarter.values for q in [1, 2, 3, 4]):
                    i1 = df[df['DT_REFER'].dt.quarter == 1].index[0]
                    i2 = df[df['DT_REFER'].dt.quarter == 2].index[0]
                    i3 = df[df['DT_REFER'].dt.quarter == 3].index[0]
                    i4 = df[df['DT_REFER'].dt.quarter == 4].index[0]

                    q1 = df[df['DT_REFER'].dt.quarter == 1]['VL_CONTA'].iloc[0]
                    q2 = df[df['DT_REFER'].dt.quarter == 2]['VL_CONTA'].iloc[0]
                    q3 = df[df['DT_REFER'].dt.quarter == 3]['VL_CONTA'].iloc[0]
                    q4 = df[df['DT_REFER'].dt.quarter == 4]['VL_CONTA'].iloc[0]
                
                    try:
                        # Perform calculations based on quarters and update flag
                        if conta_first in last_quarters:
                            if not demonstrativo_cvm.loc[i4, 'MATH_MAGIC']:
                                q4 = q4 - (q3 + q2 + q1)
                            update = True
                        elif conta_first in all_quarters:
                            if not demonstrativo_cvm.loc[i2, 'MATH_MAGIC']:
                                q2 = q2 - (q1)
                            if not demonstrativo_cvm.loc[i3, 'MATH_MAGIC']:
                                q3 = q3 - (q2 + q1)
                            if not demonstrativo_cvm.loc[i4, 'MATH_MAGIC']:
                                q4 = q4 - (q3 + q2 + q1)
                            update = True
                    except Exception as e:
                        update = False

                    # Update demo_cvm data based on calculations
                    if update:
                        demonstrativo_cvm.loc[i2, ['VL_CONTA', 'MATH_MAGIC']] = [q2, True]
                        demonstrativo_cvm.loc[i3, ['VL_CONTA', 'MATH_MAGIC']] = [q3, True]
                        demonstrativo_cvm.loc[i4, ['VL_CONTA', 'MATH_MAGIC']] = [q4, True]

                if n3 > n:
                    break
            if n2 > n:
                break
        if n1 > n:
            break
    return demo_cvm


In [ ]:
file_name = f'dataframe_{k_year}'
demo_cvm[k_year] = save_pkl(demo_cvm[k_year], file_name)

##### temp

In [ ]:
demo_cvm = load_pkl('dataframes')
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
for k_year in years:
    for y in [2010, 1011, 2012, 2013, 2014]:
        try:
            del demo_cvm[y]
        except:
            pass
    print(k_year)
    keys_to_delete = []
    for year, df in demo_cvm.items():
        if year != k_year:
            keys_to_delete.append(year)
    for key in keys_to_delete:
        del demo_cvm[key]
    demo_cvm = by_year(demo_cvm)
    
    file_name = f'dataframe_{k_year}'
    demo_cvm[k_year] = save_pkl(demo_cvm[k_year], file_name)
    demo_cvm = load_pkl('dataframes')


##### continue

In [ ]:
years = range(2010, 2024)
demo_cvm = {}
start_time = time.time()
for i, year in enumerate(years):
    print(year)
    demo_cvm[year] = load_pkl(f'dataframe_{year}')


In [ ]:
# save by company
## get an aggregated list of company, agg in all demo_cvm dict
## create a demo_cvm_company dict, transform from year to company

## must create a setor, subsetor, segmento for companies, then use as keys?

In [ ]:
# Get all unique companies across all years
all_companies = set()
for df in years:
    all_companies.update(df['company-agg'].unique())

# Initialize the final dictionary with companies as keys
company_dict = {}

# Populate the company_dict
for company in all_companies:
    company_data = []  # This will hold dataframes for each year for the company
    for year, df in year_dict.items():
        company_df_for_year = df[df['company-agg'] == company]
        company_data.append(company_df_for_year)
    
    # Concatenate the data for the company across all years
    company_dict[company] = pd.concat(company_data, ignore_index=True)


In [ ]:
df = demo_cvm[2014]

m_company = df['DENOM_CIA'] == 'CENTRAIS ELET BRAS SA  ELETROBRAS'
m_conta = df['BALANCE_SHEET'] == 'DRE'
m_conta_len = df['CD_CONTA'].str.len() <= 4
m_agg = df['AGRUPAMENTO'] == 'ind'
mask = m_company & m_conta & m_conta_len & m_agg

# Filter the DataFrame using the mask
filtered_df = df[mask][['DS_CONTA', 'DT_REFER', 'VL_CONTA']]

# Pivot the filtered DataFrame using pivot_table and aggregation function
pivot_df = filtered_df.pivot_table(index='DT_REFER', columns='DS_CONTA', values='VL_CONTA', aggfunc='sum')

# Plot the pivoted DataFrame
try:
    pivot_df.plot()
except:
    pass


In [ ]:
df['CD_CONTA'].unique()

In [ ]:
m_company = df['DENOM_CIA'] == 'ALPARGATAS SA'
m_conta = df['BALANCE_SHEET'] == 'DRE'
m_conta_len = df['CD_CONTA'].str.len() <= 4
m_agg = df['AGRUPAMENTO'] == 'ind'
mask = m_company & m_conta & m_conta_len & m_agg
df[mask]

In [ ]:
# o que caracteriza uma df completa, um relatório estruturado? DENOM_CIA + DT_REFER
# como saber quais empresas são 'con' e quais são 'ind' e como criar uma terceira view mae = con - ind?


In [ ]:
companies_by_relest

In [ ]:
# SÓ INDIVIDUAL ind = True, con = False: '3A COMPANHIA SECURITIZADORA'
# SÓ CONSOLIDADO ind = False, con = True: 'BANCO SANTANDER SA'
# AMBOS ind = True, con = True: 'ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA SA'


In [ ]:
companies_by_relest['individual']

In [ ]:
for k, v in companies_by_relest.items():
    print(k)

In [ ]:
relest_individual = True
relest_consolidado = False
pivot_table[(pivot_table['ind'] == relest_individual) & (pivot_table['con'] == relest_consolidado)].index.get_level_values('DENOM_CIA').unique()[0]

In [ ]:
cia_cols = ['CNPJ_CIA', 'DENOM_CIA', 'CD_CVM'] # 683 rows
bal_cols = ['FILE_NAME', 'demonstrativo_cvm', 'BALANCE_SHEET', 'AGRUPAMENTO', 'GRUPO_DFP'] # 32 rows
dt_cols = ['DT_REFER', 'DT_FIM_EXERC'] # 4 rows
cod_conta_cols = ['CD_CONTA', ] # 1543 rows
desc_conta_cols = ['DS_CONTA', ] # 38597 rows
vlr_cta_cols = ['VL_CONTA',]

ubiq_cols = ['ANO', 'MOEDA'] # 1 row
unique__independ_cols = ['VERSAO', 'ESCALA_MOEDA',  'ST_CONTA_FIXA', 'DT_INI_EXERC', 'COLUNA_DF']

In [ ]:
df_temp = df[cia_cols+bal_cols+dt_cols+cod_conta_cols+desc_conta_cols+vlr_cta_cols+unique__independ_cols]
mask = df_temp['CD_CVM'] == 2437
mask2 = df_temp['AGRUPAMENTO'] == 'con'
mask3 = df_temp['BALANCE_SHEET'] == 'DRE'
mask4 = df_temp['DT_REFER'].dt.month == 12
mask5 = df_temp['CD_CONTA'] == '3.01'

df_temp[mask&mask2&mask3&mask5].drop_duplicates()

In [ ]:
df

In [ ]:
# group by year

# clean df



In [ ]:
sorted_df.groupby('CD_CVM')['VERSAO'].idxmax()


In [ ]:
df = df_demo_cvm_y[year]
if 1 == 1:
    df = df.copy()  # Create a copy of the DataFrame

    # filter only last one
    df = df[df['ORDEM_EXERC'] == 'ÚLTIMO']
    df = df.drop(columns=['ORDEM_EXERC'])

    # Clean up Text
    df['DENOM_CIA'] = df['DENOM_CIA'].apply(clean_text)

    # rows to remove
    words_to_remove = ['LIQUIDACAO', 'JUDICIAL', ]
    df = df[~df['DENOM_CIA'].str.contains('|'.join(words_to_remove))]

    # DateTime
    date_columns = ['DT_REFER', 'DT_RECEB', 'DT_FIM_EXERC', ]
    # Convert the specified date columns to datetime
    for col in date_columns:
        try:
            df = df.assign(**{col: pd.to_datetime(df[col])})
        except Exception as e:
            # Handle invalid date values here
            pass

    # Sort the DataFrame by 'CD_CVM' and 'VERSAO' in descending order
    sorted_df = df.sort_values(by=['CD_CVM', 'VERSAO'], ascending=[True, False])

    # After sorting
    sorted_df = df.sort_values(by=['CD_CVM', 'VERSAO'], ascending=[True, False])
    print("After sorting:", sorted_df.shape)

    # After the groupby and idxmax operations
    indices_of_max_version = sorted_df.groupby('CD_CVM')['VERSAO'].idxmax()
    print("Indices of max version:", indices_of_max_version)


    # Get the indices of the rows with the highest version for each 'CD_CVM'
    indices_of_max_version = sorted_df.groupby('CD_CVM')['VERSAO'].idxmax()

    # Filter the DataFrame to keep only the rows with the highest version for each 'CD_CVM'
    filtered_df = sorted_df.loc[indices_of_max_version]

    df = sorted_df = df.sort_values(by=['DENOM_CIA'], ascending=[True])


    # return df.reset_index(drop=True)

In [ ]:
df_demo_cvm_y = group_by_year(df_demo_cvm, df_demo_cvm_links)

In [ ]:
df_demo_cvm_y[year]

In [ ]:
df_demo_cvm_links_y[year][df_demo_cvm_links_y[year]['CD_CVM'] == 15253].head(30)

In [ ]:
merged_df = df.merge(df_demo_cvm_links[df_demo_cvm_links['ANO'] == '2023'][cols_common + cols_to_add], on=cols_common, how='left')
merged_df

In [ ]:
df[cols_common].drop_duplicates()

In [ ]:
df_demo_cvm_links[cols_common].drop_duplicates()

In [ ]:
# save to csv

dfp.to_csv('dfp.csv')
dfp_links.to_csv('dfp_links.csv')
itr.to_csv('itr.csv')
itr_links.to_csv('itr_links.csv')


In [ ]:
# read from csv

dfp = pd.read_csv('dfp.csv')
dfp_links = pd.read_csv('dfp_links.csv')
itr = pd.read_csv('itr.csv')
itr_links = pd.read_csv('itr_links.csv')

In [ ]:
# Base URL
url_raw =        'https://www.rad.cvm.gov.br/ENET/frmGerenciaPaginaFRE.aspx?CodigoTipoInstituicao=1&NumeroSequencialDocumento={ID_DOC}'
url_download = 'http://www.rad.cvm.gov.br/ENETCONSULTA/frmDownloadDocumento.aspx?CodigoInstituicao=1&NumeroSequencialDocumento={ID_DOC}'
url_relatorio_administracao = 'https://www.rad.cvm.gov.br/ENET/frmExibirArquivoFRE.aspx?NumeroSequencialDocumento=8299&CodigoGrupo=1653&CodigoQuadro=0&Tipo=PDF&RelatorioRevisaoEspecial=Sem+Ressalva&CodTipoDocumento=4'

# Generate list of URLs
dfp_url_list = [url_raw.format(ID_DOC=id_doc) for id_doc in dfp_links['ID_DOC']]
dfp_download_list = [url_download.format(ID_DOC=id_doc) for id_doc in dfp_links['ID_DOC']]
dfp_relatorio_administracao_list = [url_relatorio_administracao.format(ID_DOC=id_doc) for id_doc in dfp_links['ID_DOC']]
itr_url_list = [url_raw.format(ID_DOC=id_doc) for id_doc in itr_links['ID_DOC']]
itr_download_list = [url_download.format(ID_DOC=id_doc) for id_doc in itr_links['ID_DOC']]
itr_relatorio_administracao_list = [url_relatorio_administracao.format(ID_DOC=id_doc) for id_doc in itr_links['ID_DOC']]


In [ ]:
def update_version_old(df, links):
    cols_to_add = ['VERSAO_DOC', 'CATEG_DOC', 'ID_DOC', 'DT_RECEB']
    
    # Create the columns if they don't exist
    for col in cols_to_add:
        if col not in df.columns:
            df[col] = None
    
    # Group the df DataFrame by the conditions
    grouped_df = df.groupby(['ANO', 'demonstrativo_cvm', 'CNPJ_CIA', 'CD_CVM', 'DT_REFER'])
    
    # Iterate through links and update grouped_df
    for i, row in links.iterrows():
        key = (row['ANO'], row['demonstrativo_cvm'], row['CNPJ_CIA'], row['CD_CVM'], row['DT_REFER'])
        group = grouped_df.get_group(key)
        
        cols_to_update = ['VERSAO', 'CATEG_DOC', 'ID_DOC', 'DT_RECEB']
        df.loc[group.index, cols_to_add] = row[cols_to_update].values
        
        print(f"{i+1}/{len(links)-i} {row['demonstrativo_cvm']} {row['ANO']} - {row['CD_CVM']} {row['CNPJ_CIA']} - {row['DT_REFER']} versão {row['VERSAO']}")
    
    while i > 20:
        print('break')
        break
    return df


In [ ]:
def update_version(df, links, grouped_df):
    cols_to_add = ['VERSAO_DOC', 'CATEG_DOC', 'ID_DOC', 'DT_RECEB']

    # Create the columns if they don't exist
    for col in cols_to_add:
        if col not in df.columns:
            df[col] = None

    # Convert columns to object type
    for col in cols_to_add:
        try:
            df[col] = df[col].astype('object')
        except Exception as e:
            pass

    for i, key in enumerate(grouped_df.groups.keys()):
        print(f'{i}/{len(grouped_df)} {key}')
        
        grouped_df.get_group(key)
        ano, demonstrativo_cvm, cnpj_cia, cd_cvm, dt_refer = key

        # Filter links based on the group's conditions
        mask = (
            (links['ANO'] == ano) &
            (links['demonstrativo_cvm'] == demonstrativo_cvm) &
            (links['CNPJ_CIA'] == cnpj_cia) &
            (links['CD_CVM'] == cd_cvm) &
            (links['DT_REFER'] == dt_refer)
        )


        df.loc[grouped_df.get_group(key).index, cols_to_add] = links[mask][cols_to_add].values

    # Convert columns to object type
    for col in cols_to_add:
        try:
            df[col] = df[col].astype('category')
        except Exception as e:
            pass

    return df


In [ ]:
# Group the df DataFrame by the conditions
group_columns = ['ANO', 'demonstrativo_cvm', 'CNPJ_CIA', 'CD_CVM', 'DT_REFER']
grouped_df = itr.groupby(group_columns)

itr = update_version(itr, itr_links, grouped_df)


# # Group the df DataFrame by the conditions
# group_columns = ['ANO', 'demonstrativo_cvm', 'CNPJ_CIA', 'CD_CVM', 'DT_REFER']
# grouped_df = dfp.groupby(group_columns)

# dfp = update_version(dfp, dfp_links, grouped_df)